In [53]:
# Dependencies
import numpy as np
import pandas as pd

In [54]:
censuses_data = pd.read_csv("../../CSV/censuses.csv")
# censuses_data = censuses_data[censuses_data["census_year"] == 1900]
censuses_data.count()

line_number               4244
house_number              4090
street_name               4090
last_name                 4090
given_name                4090
relation                  4090
color_race                4244
sex                       4244
age_at_last_birthday      4244
occupation                4244
own_or_rent               4090
owned_free_or_mortgage    4090
house_or_farm             4090
my_notes                  4090
file_name                 4090
census_year               4244
grouped_occu              4244
dtype: int64

In [42]:
# # Separate out the occupation which are null/ none (LATER we can test our model again these cases)
# occupation_missing_mask = (censuses_data["occupation"].str.contains('None')) | (censuses_data["occupation"].str.contains('Null'))
# df_missing_occupation = censuses_data[occupation_missing_mask]

# df_missing_occupation.count()

In [43]:
# # prepare new df where occupation column is All labelled
# df_all_occupation = censuses_data[occupation_missing_mask == False]

# # include following features that MAY influence occupation field -
# # color_race, sex, age_at_last_birthday, occupation
# newdata = df_all_occupation.loc[:,["color_race","sex","age_at_last_birthday", "occupation"]]
# newdata.count()

In [46]:
newdata = censuses_data.loc[:,["color_race","sex","age_at_last_birthday", "census_year", "grouped_occu"]]
newdata.count()

color_race              4244
sex                     4244
age_at_last_birthday    4244
census_year             4244
grouped_occu            4244
dtype: int64

In [49]:
# newdata.groupby(['occupation']).count().sort_values(by=['occupation'])
newdata.groupby(['census_year']).grouped_occu.value_counts()
# newdata.sort_values('occupation', ascending=False)

census_year  grouped_occu         
1900         Missing Occupations       321
             Unskilled                 277
             Student                   121
             Skilled Labor              84
             Skilled Professionals       5
1910         Unskilled                 116
             Skilled Labor              20
             Missing Occupations        14
             Skilled Professionals       4
1920         Unskilled                1580
             Missing Occupations      1068
             Skilled Labor             503
             Skilled Professionals     131
Name: grouped_occu, dtype: int64

In [51]:
def age_clean(data,under_1_value = 1, pandas = 'yes'):
    clean_age = []
    if pandas == 'yes':
        data_lst = data.tolist()
    else:
        data_lst = data
    for datum in data_lst:
        if 'month' in str(datum):
            if 'and' in str(datum):
                clean_age.append(str(datum).split(' ')[0])
            else:
                clean_age.append(str(under_1_value))
        else:
            clean_age.append(datum)
    return clean_age

In [52]:
# Cleaning up age
newdata.age_at_last_birthday = age_clean(newdata.age_at_last_birthday)

0       26
1       27
2       25
3       25
4        8
5       46
6       34
7       19
8       15
9       12
10       9
11       8
12      21
13       5
14       5
15       0
16       2
17      58
18      35
19      23
20      18
21      13
22      56
23      41
24      20
25      39
26      20
27      19
28       2
29       7
        ..
4214    24
4215    22
4216    20
4217    36
4218    28
4219    50
4220    18
4221    16
4222    39
4223    20
4224    29
4225    30
4226    30
4227    50
4228    44
4229    24
4230    50
4231    50
4232    25
4233    34
4234    21
4235    37
4236    31
4237    27
4238    40
4239    19
4240    38
4241    22
4242    27
4243    26
Name: age_at_last_birthday, Length: 4244, dtype: object
##################################################
0       26
1       27
2       25
3       25
4        8
5       46
6       34
7       19
8       15
9       12
10       9
11       8
12      21
13       5
14       5
15       0
16       2
17      58
18      35
19      23
20

In [7]:
from fuzzywuzzy import fuzz, process

In [34]:
correct_occupations=["Agent","Barber","Bakery","Beauty","Cook","Clerk","Cleaner","Drive","Express", "Fireman","Foreman","Helper",\
                     "Labor","Laundr","Machinist","Manager","Maid",\
                     "Paint","Plumber","Porter","Railway","Teamster","Wait","Washwoman","Worker", "Wringer","Yard" ]
# replace Yard by Yardman
# replace wait by waiter/ Waitress
# replace Laundr by Laundress

In [35]:
def fuzzy_match_occ(occupation):
    global df_occu
    new_name, score = process.extractOne(occupation, correct_occupations)
    if(score > 70):
        # Good match case
        if(new_name in df_occu):
        # entry found in corrected occupation then append it
            if not(occupation in df_occu[new_name]):
                df_occu[new_name].append(occupation)
        else:
            # make entry for the new_name found
            df_occu[new_name] = [occupation,]
    else:
        # Poor match score 
        if not(occupation in df_occu):
            # make entry for the new_name found
            df_occu[occupation] = [occupation,]
    
    return df_occu

In [36]:
df_occu = dict()
data = zip(*newdata['occupation'].apply(fuzzy_match_occ))

In [37]:
type(newdata['occupation'])

pandas.core.series.Series

In [38]:
print(len(df_occu))
print("*****************************************")
for row in df_occu:
    print(row, df_occu[row])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print("Matching similar occupation count is ", len(df_occu[row]))
    print("---------------------------------")

509
*****************************************
Servant ['Servant']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Drayman ['Drayman']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Laundr ['Laundress', 'Laundryman', 'Landlord', 'Ironer At Laundry', 'Washwoman At Laundry', 'Wringer Man At Laundry', 'Laundress At Home', 'Laundress At Private Families', 'Starcher At Laundry', 'Sollicitor At Laundry', 'Laundress At Private Family', 'Show Boss At Laundry', 'Washer At Laundry', 'Laundress At Hotel', 'Dresser At Laundry', 'Sorter At Laundry', 'Market At Laundry', 'Stableman At Laundry', 'Worker At Laundry', 'Folder At Laundry', 'Porter At Laundry', 'Presser At Laundry', 'Finisher At Laundry', 'Wringer At Laundry', 'Marker At Laundry', 'Presser At Lauyndry', 'Shirt Finisher At Laundry', 'Laundress At Laundry', 'Finisher At Lauyndry', 'Stamper At Laundry', 'Seamstress At Laun

Operator At Elevator ['Operator At Elevator']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Newsboy At M Paper Company ['Newsboy At M Paper Company']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Harness Maker For Harness Company ['Harness Maker For Harness Company']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Bookkeeper At Harness Company ['Bookkeeper At Harness Company']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Veterinary Doctor ['Veterinary Doctor']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Owner At Rooming Home ['Owner At Rooming Home']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Miller At New Mill ['Mil

Matching similar occupation count is  1
---------------------------------
Dealer At Fish And Oysters ['Dealer At Fish And Oysters']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Merchant At Grocery Store ['Merchant At Grocery Store']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Pastor At Colored Baptist Church ['Pastor At Colored Baptist Church']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Farmer At Cotton And Corn ['Farmer At Cotton And Corn']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Glassier (Retired) At Buildings ['Glassier (Retired) At Buildings']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Messman At Steamboat Company ['Messman At Steamboat Company']
~~~~~~~~~~~~~~~~

Help At Domestic ['Help At Domestic']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Janitor At School ['Janitor At School']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Shiner At Shoes ['Shiner At Shoes']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Domestic Work Private Families ['Domestic Work Private Families']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Mechanic At Sugar Refinery ['Mechanic At Sugar Refinery']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Janitor At Telephone Co ['Janitor At Telephone Co']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Chauffeur At Grocery Store ['Chauffeur At Grocery Store']
~~~~

In [8]:
def fuzziest_match (inp, choices_inp, scorer = 'all', match_score = 75, results = 10, pandas = 'yes'):
    #Set up
    possibilities = {}
    #Pandas Handling
    if pandas == 'yes':
        choices = choices_inp.tolist() #ASK JUSTIN
    else:
        choices = choices_inp
    #Score dictionary for fuzz methods
    score_switch = {
        'all': [fuzz.ratio,fuzz.partial_ratio,fuzz.token_sort_ratio,fuzz.WRatio],
        'ratio': fuzz.ratio,
        'partial': fuzz.partial_ratio,
        'token': fuzz.token_sort_ratio,
        'WRatio': fuzz.WRatio}
    #Score Handler
    if scorer == 'ratio':
        scorer_opt = score_switch['ratio']
    elif scorer == 'partial':
        scorer_opt = score_switch['partial']
    elif scorer == 'token':
        scorer_opt = score_switch['token']
    elif scorer == 'WRatio':
        scorer_opt = score_switch['WRatio']
    elif scorer not in score_switch.keys():
        raise Exception ('Invalid Scorer')
        print(scorer +' is not a valid score option')
        print('/n')
        print('Options: ')
        for key in score_switch:
            print(key, score_switch[key])
    #Nested DRY functions
    def inner_fuzz (inp, choices, scorer,score_switch):
        matches = process.extract(
            inp,
            choices,
            limit = results,
            scorer = scorer
            )
        return matches
    
    def all_inner_fuzz (inp, choices, score_switch):
        for option in score_switch['all']:
            matches = process.extract(
                inp,
                choices,
                limit = results,
                scorer = option
                )
        return matches

    #Type handling    
    if type(inp) == list:
        if type(inp[0]) == list:
            for lst in inp:
                for lst_item in lst:
                    if scorer == 'all':
                        possibilities[lst_item] = all_inner_fuzz(lst_item,choices,score_switch)
                    else:
                        possibilities[lst_item] = inner_fuzz(lst_item,choices,scorer_opt,score_switch)
            return possibilities
        else:
            for item in inp:
                if scorer == 'all':
                    possibilities[item] = all_inner_fuzz(item,choices,score_switch)
                else:
                    possibilities[item] = inner_fuzz(item,choices,scorer_opt,score_switch)
        return possibilities

In [ ]:
skilled_labor=fuzziest_match(['coachman','expressman','baker','band leader','barber','beauty','blacksmith','boiler'\
                             'boilermaker','book keeper','brick','cashier','carpenter','chef','cook','conductor','fireman',\
                             'farmer','grocer','hairdresser','jeweller','tailor','manager','manicure','mechanic','mid-wife',\
                             'musician','nurse','office','operator','performer','photographer','plaster','plumber','repairman',\
                             'salesman','saleslady','secretary','seamstress','stenographer','tailor','well driller','teacher',\
                             'teamster'], newdata['occupation'], scorer = 'all', results = 2000)
student=fuzziest_match(['school'], newdata['occupation'], scorer = 'all', results = 2000)
unskilled=fuzziest_match(['servant','laundress','drayman','labor',], newdata['occupation'], scorer = 'all', results = 2000)


In [13]:
skilled_prof=fuzziest_match(['Abstracter','Accountant','Actor','Actress','Agent','Manager','Chemist','Chief','Owner',\
                            'Contracter','Operator','Dealer','Doctor','Landlord','Lawyer','Live Stock Buyer','Merchant',\
                            'Owner','Physician','Boss','Surgeon','Soldier','Proprietor'], newdata['occupation'], scorer = 'all', results = 2000)

# skilled_prof=fuzziest_match(['soldier'], newdata['occupation'], scorer = 'all', results = 2000)


In [24]:
skilled_prof['Actress']

[('Actress At Theatrical Company', 90),
 ('Waitress At Coffee', 77),
 ('Waitress At Caffee', 77),
 ('Waitress At Caffee', 77),
 ('Waitress At Boarding House', 77),
 ('Waitress At Restaurant', 77),
 ('Waitress At Caf\x8e', 77),
 ('Waitress At Cafe', 77),
 ('Waitress At Restaurant', 77),
 ('Waitress At Cvafe', 77),
 ('Waitress At Rooming House', 77),
 (' Waitress At Restaurant', 77),
 ('Proprietor At Mattress Factory', 77),
 ('Salesman At Mattress Co', 77),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Seamstress', 71),
 ('Cook At Restaurant ', 64),
 ('Dishwasher At Restaurant', 64),
 ('Yard Man At Restaurant', 64),
 ('Bus Boy At Restaurant', 64),
 ('Bus Boy At Restaurant', 64),
 ('Cook At Restaurant ', 64),
 ('Waiter At Restaurant', 64),
 ('Waiter At Restaurant', 64),
 ('Bus Boy At Restaurant', 64),
 ('Cleaner At Pressing Shop', 64),
 ('Seamstress At Home', 64),
 